1. Read in the provided data file using pandas and convert missing data to NA values.

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

data = pd.read_csv("~/work/exercises/final-exercise-hennaylimaa/data/2315676.txt", na_values=-9999, skiprows=[1], delim_whitespace=True)
data.head()

,STATION,STATION_NAME,DATE,TAVG,TMAX,TMIN
0,GHCND:FI000007501,SODANKYLA-AWS-FI,19080101,NaN,2.0,-37.0
1,GHCND:FI000007501,SODANKYLA-AWS-FI,19080102,NaN,6.0,-26.0
2,GHCND:FI000007501,SODANKYLA-AWS-FI,19080103,NaN,7.0,-27.0
3,GHCND:FI000007501,SODANKYLA-AWS-FI,19080104,NaN,-3.0,-27.0
4,GHCND:FI000007501,SODANKYLA-AWS-FI,19080105,NaN,4.0,-36.0


2. Fill in the missing values in the `TAVG` column with your estimates of the average daily temperature as noted above in the data section. You can then drop any average daily temperatures that are still missing.

In [48]:
data["AVG"] = data[['TMAX', 'TMIN']].mean(axis=1) #(data["TMAX"] + data["TMIN"]) / 2
data.loc[data['TAVG'].isna() == True, 'TAVG'] = data['AVG']
data.head()

,STATION,STATION_NAME,DATE,TAVG,TMAX,TMIN,AVG
0,GHCND:FI000007501,SODANKYLA-AWS-FI,19080101,-17.5,2.0,-37.0,-17.5
1,GHCND:FI000007501,SODANKYLA-AWS-FI,19080102,-10.0,6.0,-26.0,-10.0
2,GHCND:FI000007501,SODANKYLA-AWS-FI,19080103,-10.0,7.0,-27.0,-10.0
3,GHCND:FI000007501,SODANKYLA-AWS-FI,19080104,-15.0,-3.0,-27.0,-15.0
4,GHCND:FI000007501,SODANKYLA-AWS-FI,19080105,-16.0,4.0,-36.0,-16.0
